# 🍿 Snowflake Trail - Step 1: Demo Setup

Creating a dummy data ingestion and transformation project with some intentional errors and anomalies that we can monitor and alert on.

* Warehouse
* Internal Stage
* Pipe
* Dynamic Tables
* Stream and Task graph
* nested SQL & Python Procedures & Functions

---
**Setup Details**:
* 1st Task creates a new csv-file in the internal Stage
* 2nd Task creates a broken csv-file into the internal stage
* 3rd Task manually refreshes the Pipe to load the 2 new files from the source Stage into a target table
* Pipe copy of the broken csv-file will fail -> logging file ingestion error to the event table
* the Stream on the target table will catch the new data from the successful file ingestion
* Stream triggers the root Task to run the graph
* child Tasks call procedures and functions, which may fail at random -> logging Procedure and Task errors
* parallel child Task will change the name of the source table of a Dynamic Table and then manually refresh the Dynamic Table -> logging Dynamic Table refresh error

-> manually run the Trigger Task via the UI or SQL to create new logs

In [ ]:
create database if not exists SNOWTRAIL_DEMO;

In [ ]:
-- just to clean up 
drop schema if exists SNOWTRAIL_DEMO.PUBLIC;

In [ ]:
-- schema to contain the business data transformation process
create schema if not exists SNOWTRAIL_DEMO.PIPELINE;

In [ ]:
-- warehouse to run the data transformation process 
create warehouse if not exists SNOWTRAIL_PIPELINE_WH
    warehouse_size = XSMALL
    auto_suspend = 300                      -- after 5min
    statement_timeout_in_seconds = 1800     -- after 30min
    comment = 'running only data transformations'
;

In [ ]:
create or replace stage SNOWTRAIL_DEMO.PIPELINE.WEATHER_CSV_FILES 
directory = ( enable = true ) 
comment = 'to create csv files for pipe ingestion'
;

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.TRIGGER_ERROR_LOGS
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment = 'manually run task to trigger new event logs'
as
    select SYSTEM$WAIT(2)
;

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.INSERT_NEW_FILE
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment = 'write csv files into stage'
after
    SNOWTRAIL_DEMO.PIPELINE.TRIGGER_ERROR_LOGS
as
declare
    TS varchar := replace(concat(current_date(),'-',current_time()), ':','-');
begin
    execute immediate '
    copy into 
        @SNOWTRAIL_DEMO.PIPELINE.WEATHER_CSV_FILES/'|| :TS ||'/BROKEN
    from (
        select 
            current_date() as DS,
            ''Germany'' as COUNTRY,
            13184 as ZIPCODE, 
            42 as TEMP_IN_F,
            15 as WIND_IN_MPH, 
            1 as RAIN_IN_INCH
        ) 
    FILE_FORMAT = (TYPE = CSV)
    OVERWRITE = TRUE
';
end;

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.INSERT_ANOMALY_FILE
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment = 'write csv files into stage'
after
    SNOWTRAIL_DEMO.PIPELINE.INSERT_NEW_FILE
as
declare
    TS varchar := replace(concat(current_date(),'-',current_time()), ':','-');
begin
    execute immediate '
        copy into 
            @SNOWTRAIL_DEMO.PIPELINE.WEATHER_CSV_FILES/'|| :TS ||'/QUALITY_ISSUE
        from (
            select 
                current_date() as DS,
                ''Germany'' as COUNTRY,
                13184 as ZIPCODE, 
                142 as TEMP_IN_F,
                15 as WIND_IN_MPH, 
                1 as RAIN_IN_INCH
            ) 
        FILE_FORMAT = (TYPE = CSV)
        OVERWRITE = TRUE
    ';
end;

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.INSERT_BROKEN_FILE
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment = 'write incomplete csv files into stage'
after
    SNOWTRAIL_DEMO.PIPELINE.INSERT_NEW_FILE
as
declare
    TS varchar := replace(concat(current_date(),'-',current_time()), ':','-');
begin
    execute immediate '
        copy into 
            @SNOWTRAIL_DEMO.PIPELINE.WEATHER_CSV_FILES/'|| :TS ||'/BROKEN
        from (
            select 
                current_date as DS,
                ''Germany'' as COUNTRY,
                13184 as ZIPCODE, 
                15 as WIND_IN_MPH, 
                1 as RAIN_IN_INCH
            ) 
        FILE_FORMAT = (TYPE = CSV)
        OVERWRITE = TRUE
    ';
end;

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.MAN_PIPE_REFRESH
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment = 'manually refresh pipe to pick up new files'
after
    SNOWTRAIL_DEMO.PIPELINE.INSERT_BROKEN_FILE
as
    alter pipe SNOWTRAIL_DEMO.PIPELINE.LOAD_DAILY_WEATHER refresh
;

In [ ]:
alter task SNOWTRAIL_DEMO.PIPELINE.INSERT_NEW_FILE resume;
alter task SNOWTRAIL_DEMO.PIPELINE.INSERT_ANOMALY_FILE resume;
alter task SNOWTRAIL_DEMO.PIPELINE.INSERT_BROKEN_FILE resume;
alter task SNOWTRAIL_DEMO.PIPELINE.MAN_PIPE_REFRESH resume;

In [ ]:
create or replace table SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER (
	DS DATE,
	COUNTRY VARCHAR(16777216),
	ZIPCODE VARCHAR(16777216),
	TEMP_IN_F NUMBER(38,0),
	WIND_IN_MPH NUMBER(38,0),
	RAIN_IN_INCH NUMBER(38,0)
)
comment = 'Pipe target table'
;

In [ ]:
create or replace pipe SNOWTRAIL_DEMO.PIPELINE.LOAD_DAILY_WEATHER
comment = 'requires manual loading'
as
copy into 
    SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER
from 
    @SNOWTRAIL_DEMO.PIPELINE.WEATHER_CSV_FILES   
file_format = (type = CSV)
on_error = continue
;

In [ ]:
create or replace dynamic table SNOWTRAIL_DEMO.PIPELINE.ALL_WEATHER_IMPERIAL
target_lag = 'DOWNSTREAM'
warehouse = 'SNOWTRAIL_PIPELINE_WH'
as
select
    DS,
    COUNTRY,
    ZIPCODE,
    TEMP_IN_F,
    WIND_IN_MPH,
    RAIN_IN_INCH
from 
    SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER
group by
    ALL
order by 
    DS desc
;

In [ ]:
create or replace dynamic table SNOWTRAIL_DEMO.PIPELINE.ALL_WEATHER_METRIC
target_lag = '8 hours' 
warehouse = 'SNOWTRAIL_PIPELINE_WH'
as
select 
    DS,
    COUNTRY,
    ZIPCODE,
    ((TEMP_IN_F - 32) * 5/9) AS TEMP_IN_C,
    (WIND_IN_MPH * 1.609344) AS WIND_IN_KMH,
    (RAIN_IN_INCH * 2.54) AS RAIN_IN_CM
from  
    SNOWTRAIL_DEMO.PIPELINE.ALL_WEATHER_IMPERIAL
;

In [ ]:
create or replace stream SNOWTRAIL_DEMO.PIPELINE.NEW_WEATHER_DATA 
on table SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER
append_only = true
comment = 'to trigger Task';

In [ ]:
create or replace table SNOWTRAIL_DEMO.PIPELINE.NEW_US_WEATHER (
    DS DATE,
	COUNTRY VARCHAR(16777216),
	ZIPCODE VARCHAR(16777216),
	TEMP_IN_F NUMBER(38,0),
	WIND_IN_MPH NUMBER(38,0),
	RAIN_IN_INCH NUMBER(38,0)
);

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.ROOT_TASK
warehouse = 'SNOWTRAIL_PIPELINE_WH'
-- no scheduled -> triggered
SUSPEND_TASK_AFTER_NUM_FAILURES = 0
when
    SYSTEM$STREAM_HAS_DATA('SNOWTRAIL_DEMO.PIPELINE.NEW_WEATHER_DATA')
as
insert into SNOWTRAIL_DEMO.PIPELINE.NEW_US_WEATHER
    select 
        DS,
    	COUNTRY,
    	ZIPCODE,
    	TEMP_IN_F,
    	WIND_IN_MPH,
    	RAIN_IN_INCH
    from
        SNOWTRAIL_DEMO.PIPELINE.NEW_WEATHER_DATA    -- new rows in stream
;

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.COPY_FROM_STREAM
warehouse = 'SNOWTRAIL_PIPELINE_WH'
after
    SNOWTRAIL_DEMO.PIPELINE.ROOT_TASK
as
insert into SNOWTRAIL_DEMO.PIPELINE.NEW_US_WEATHER
    select 
        DS,
    	COUNTRY,
    	ZIPCODE,
    	TEMP_IN_F,
    	WIND_IN_MPH,
    	RAIN_IN_INCH
    from
        SNOWTRAIL_DEMO.PIPELINE.NEW_WEATHER_DATA    -- new rows in stream
;

In [ ]:
create or replace procedure SNOWTRAIL_DEMO.PIPELINE.SQL_PROCEDURE()
returns VARCHAR(16777216)
language SQL
execute as OWNER
as 
$$
begin
    SYSTEM$LOG_INFO('My SQL procedure started');
    select SYSTEM$WAIT(2);
    SYSTEM$LOG_INFO('My SQL procedure completed');
end
$$;

In [ ]:
create or replace procedure SNOWTRAIL_DEMO.PIPELINE.PYTHON_PROCEDURE()
returns VARCHAR(16777216)
language PYTHON
RUNTIME_VERSION = '3.9'
PACKAGES = ('snowflake-snowpark-python','snowflake-telemetry-python')
HANDLER = 'wait_5_seconds'
execute as OWNER
as
$$
import time
import logging
from snowflake import telemetry

logger = logging.getLogger("DEMO_LOGGER")

def wait_5_seconds(session):
    logger.info("My Python procedure started")
    time.sleep(5)
    logger.warn("My Python procedure completed late")
$$;

In [ ]:
create or replace procedure SNOWTRAIL_DEMO.PIPELINE.JAVASCRIPT_PROCEDURE()
returns string
language javascript
as
$$
    snowflake.log("info", "My JavaScript procedure started");      
    snowflake.log("warn", "My JavaScript procedure completed late");      
    return "Done";
$$;

In [ ]:
create or replace procedure SNOWTRAIL_DEMO.PIPELINE.FLAKY_SQL_PROCEDURE()
returns VARCHAR(16777216)
language SQL
execute as OWNER
as 
$$
declare
    RANDOM_VALUE number(2,0);
begin
    RANDOM_VALUE := (select uniform(1, 2, random()));
    if (:RANDOM_VALUE = 2) 
        then select count(*) from OLD_TABLE;
    end if;
    select SYSTEM$WAIT(2);
end
$$;

In [ ]:
create or replace function SNOWTRAIL_DEMO.PIPELINE.PYTHON_UDF()
returns VARCHAR(16777216)
language PYTHON
RUNTIME_VERSION = '3.9'
PACKAGES = ('snowflake-snowpark-python','snowflake-telemetry-python')
HANDLER = 'main'
AS '
import time
import logging
from snowflake import telemetry   
logger = logging.getLogger("DEMO_LOGGER")

def main():
    time.sleep(5)
    logger.info("Python UDF completed.")
    return "Python UDF completed."

';

In [ ]:
create or replace procedure SNOWTRAIL_DEMO.PIPELINE.PARENT_PROCEDURE()
returns VARCHAR(16777216)
language SQL
execute as OWNER
as 
$$
begin
    call SNOWTRAIL_DEMO.PIPELINE.SQL_PROCEDURE();
    call SNOWTRAIL_DEMO.PIPELINE.PYTHON_PROCEDURE();
    call SNOWTRAIL_DEMO.PIPELINE.JAVASCRIPT_PROCEDURE();
    call SNOWTRAIL_DEMO.PIPELINE.FLAKY_SQL_PROCEDURE();
    SYSTEM$LOG_INFO('Demo parent procedure completed');
end
$$;

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.CALL_PROCEDURES
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment='calling procedures'
after
    SNOWTRAIL_DEMO.PIPELINE.ROOT_TASK
as
begin
    call SNOWTRAIL_DEMO.PIPELINE.PARENT_PROCEDURE();
    select SYSTEM$WAIT(2);
    select SNOWTRAIL_DEMO.PIPELINE.PYTHON_UDF();
    select SYSTEM$WAIT(2);
    call SYSTEM$SET_RETURN_VALUE('Task 2 completed');
end
;       

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.MAKE_DT_REFRESH_FAIL
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment='changing base table name and refreshing DT'
after
    SNOWTRAIL_DEMO.PIPELINE.ROOT_TASK
as
begin
    alter table SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER rename to SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER_2;
    alter dynamic table SNOWTRAIL_DEMO.PIPELINE.ALL_WEATHER_IMPERIAL refresh;
end
;       

In [ ]:
create or replace task SNOWTRAIL_DEMO.PIPELINE.CLEANUP
warehouse = 'SNOWTRAIL_PIPELINE_WH'
comment='changing base table name back to original'
finalize = SNOWTRAIL_DEMO.PIPELINE.ROOT_TASK
as
begin
    alter table if exists SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER_2 rename to SNOWTRAIL_DEMO.PIPELINE.IMPORTED_WEATHER;
    alter dynamic table SNOWTRAIL_DEMO.PIPELINE.ALL_WEATHER_IMPERIAL refresh;
end
;   

In [ ]:
select SYSTEM$TASK_DEPENDENTS_ENABLE('SNOWTRAIL_DEMO.PIPELINE.ROOT_TASK');

In [ ]:
-- requires GRANT MODIFY LOG LEVEL ON ACCOUNT TO ROLE XXX;
 
alter schema SNOWTRAIL_DEMO.PIPELINE 
    set LOG_LEVEL = INFO;
    

In [ ]:
-- requires GRANT MODIFY TRACE LEVEL ON ACCOUNT TO ROLE XXX;

alter schema SNOWTRAIL_DEMO.PIPELINE 
    set TRACE_LEVEL = ALWAYS;

Now you can intentionally trigger a range of event logs from

**Task run, Pipe copy, Dynamic Table refresh, Snowpark Procedure and Function calls**

to test or demo the logging to the event table.

In [ ]:
-- manually trigger this task to create error logs

execute task SNOWTRAIL_DEMO.PIPELINE.TRIGGER_ERROR_LOGS;

continue with the 

**Snowflake Trail - Step 2: Observability Setup** Notebook 

to add a comprehensive observability-layer to this pipeline.